<h1>Victorian Literature Project</h1>
<h3>Zirak, Aleksandr, Jialin</h3>

<p>This notebook cotains all the code for the Data Science II final project</p>

<p>Our topic was Victorian Literature where we are to extract different themes from the works of different Victorian-era writers. The data is pulled from the GitHub repository of Project Gutenberg at https://github.com/DigiUGA/Gutenberg_Text</p>

<p>The biggest challenge that was presented was quantifying what it means for a word to be part of a theme. Since our data was so large, we had to stay away from supervised learning methods since labelling large amounts of data was unrealistic. We then settled on solving the problem with a semi-supervised approach where we would manually label portions of the data to help the model learn themes, and then have it generalize to other words in the text corpus.</p>

<p>Our approach starts with pulling the dataset from GitHub. </p>

</p>We then passed the text through several filters created with NLTK. These filters include removing puncutation, tokenizing all the words, conversion to lowercase, removing stop words pulled from two libraries (aproximately 1050 words), and finally removing all numbers. We tried to apply stemming which would find the root of a word, but felt that it did not sufficiently consolidate the total vocabulary size. The words were finally grouped in a set which constricted it to only unique words, of which there were approximately </p>

<p>Next, we loaded the pretrained GloVe embedding obtained from Stanford. The embedding was trained on 840 billion tokens and mapped approximately 200 million unique words to 300 dimensions. Then, for every single word in our vocabulary set, we found the resulting embedding and stacked an embedding for each word into a matrix while also noting which word appeared on which row.</p>


<p>With an mapping of all relavent vocabulary words that appear in our dataset to their embeddings, the resulting matrix was run through the radial basis function kernel that generated an affinity matrix. Fortunately, because the difference between word A and word B is the same as the difference between word B and word A, the resulting graph after the kernel function is symmetric which simplifies the MRW</p>

<p>The MultiRank Walk then takes the affinity matrix and our labelled words and generates a series of u vectors which can be used to cluster each word in our vocab set into our predefined classes. Now, the text of different Victorian authors can be clustered into different categories.</p>

# Global Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import rbf_kernel

import nltk
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.corpus import stopwords

from glob import glob
import os
import sys
import wget
from urllib import request
import zipfile

%matplotlib inline

<h1>Text Corpus preprocessing</h1>

<p>Downloads necessary texts and NLTK packages. Reads all the files and generates tokens which are then passed through a series of filters that remove punctuation, numbers, stop words, converts to lower case, and removes duplicates.</p>

In [ ]:
# Cloning repository containing all the text files
!git clone https://github.com/YJL0629/English-Writers

In [ ]:
# Downloading necessary collections of stop words that will be removed from text
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
"""
Consolidating text files which will make up our text corpus. 
"""

raw = ""

for file in glob("./English-Writers/*/*"):
    with open(file, 'r') as f:
        raw += f.read().replace('\n', ' ')

In [184]:
"""
Parsing raw text and preprocessing into a workable set.
"""

def process_text(text):
    
    # Tokenizing words.
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    # Sending words to lowercase and removing duplicates.
    vocab = [w.lower() for w in tokens]

    # Generating list of "stop" words.
    stop_words = list(get_stop_words('en'))         #About 900 stopwords
    nltk_words = list(stopwords.words('english')) #About 150 stopwords
    stop_words.extend(nltk_words)

    # Removing unecessary "stop" words.
    vocab_wo_stopwords = [w for w in vocab if not w in stop_words]

    # Removing numbers.
    vocab = [word for word in vocab_wo_stopwords if word.isalpha()]

    return vocab

In [ ]:
# Removing duplicates.
vocab_set = process_text(raw)

# Logging.
print("=" * 20)
print("The set of the vocab words has %d unique words" % len(vocab_set))
print("=" * 20)

<h1>Obtaining Embedding Data</h1>

<p>This portion of the notebook pulls the embeddings from Stanford and unzips them into a folder called 'embedding'.</p>

<p>Note- this takes a long time.</p>

In [ ]:
# Url that contains the embeddings.
url = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'

# Download.
wget.download(url, './glove.840B.300d.zip')

In [ ]:
# Unzipping.
with zipfile.ZipFile('./glove.840B.300d.zip', 'r') as zip_ref:
    zip_ref.extractall('./embedding')

<h1>Generating Matrix of Embeddings</h1>

<p>Generates a matrix where each row is a word and the columns are the word's embedding in 300 dimensions. Words are only added to this matrix if they appear in the vocabulary set generated by parsing Victorian text.</p>

In [ ]:
# Path to embedding .txt file.
glove_dir = './embedding'

# Maps the word to its embedding (vector).
word_to_embedding = {}

# Final matrix that is (len(vocab_set) x len(embedding)).
full_embedding = None

embedding_matrix = None
word_to_matrix_row = {}

# Looping through all rows of the embeddings.
with open(os.path.join(glove_dir, 'glove.840B.300d.txt')) as f:
    
    index_in_matrix = 0
    embedding_line = 0
    
    # For each word and its embedding.
    for line in f:

        # Logging.
        embedding_line += 1
        sys.stdout.write("\r%d" % embedding_line)
        
        # Splitting the embedding line.
        values = line.split()
        
        # The first value is the word, the rest are the values of the embedding.
        word = values[0] 

        # If a word that has an embedding does not appear in the vocab set .
        if word not in vocab_set:
            continue
        
        try:
            # Read embedding for that word.
            embedding = np.asarray(values[1:], dtype='float32')

        # Some embeddings have unusual formatting.
        except Exception:
            print("\nCorrupted embedding for %s" % word)
            continue
            
        # If no rows have been added to the matrix of the embeddings.
        if embedding_matrix is None:
            
            # Initialize the embedding matrix.
            embedding_matrix = np.array([embedding])
            
            # Map the word to an index in the matrix.
            word_to_matrix_row[word] = index_in_matrix
            index_in_matrix += 1
            
        # If this word contains a proper embedding.
        elif len(embedding) == 300:
            
            # Add the embedding to our matrix.
            embedding_matrix = np.vstack((embedding_matrix, embedding))
            
            # Map the word to an index in the matrix.
            word_to_matrix_row[word] = index_in_matrix
            index_in_matrix += 1
            
        else:
            print("\nCould not add %s to the embedding matrix" % word)
            
# Set of our vocab words that are represented in the embedding matrix,
# will be smaller that vocab_set because some words do not have an embedding.
represented_vocab_set = set(word_to_matrix_row.keys())

<h1>Checkpoint</h1>

<p>Saving the mapping of words to which line line of the embedding matrix they appear in.</p>

In [ ]:
np.save('Checkpoints/word_to_matrix_row', word_to_matrix_row)

<h1>Logging Results Conversion of Vocab to Embedding Matrix</h1>

In [ ]:
# Info about original vocab set and the new one after removing words
# that did not have an embedding.
print("=" * 20)
print("The original vocab had %d words" % len(vocab_set))
print("The updated vocab set has %d words" % len(represented_vocab_set))
print("=" * 20)

# Words that are in the vocab set but did not make it into the embedding matrix
# may be due to the embedding being corrupted or an embedding for that word may not exist.
print("=" * 20)
print("%d words have been thrown out due to not having an embedding" % (len(vocab_set) - len(represented_vocab_set)))
print("=" * 20)

# Dimension of the embedding matrix.
embedding_matrix_shape = np.shape(embedding_matrix)

print("=" * 20)
print("The embedding matrix is %d by %d" % (embedding_matrix_shape[0], embedding_matrix_shape[1]))
print("=" * 20)

<h1>Generating Affinity Matrix</h1>

<p>Uses the embedding matrix to generate an affinity matrix by using the radius basis function kernel. This displays a graph of the relationships between words.</p>

In [ ]:
# Using the Radial Basis Function kernel to generate an affinity matrix.
affinity_matrix = rbf_kernel(X=embedding_matrix, gamma=0.01)

plt.imshow(affinity_matrix)

<h1>Checkpoint</h1>

<p>Saving affinity matrix so that it will not be computed later.</p>

In [ ]:
np.save('Checkpoints/affinity_matrix', affinity_matrix)

<h1>Generating Dataset</h1>

<p>Loading hand-labelled samples from text files on the disk and giving each of the classes a unique id. All other words that appear in the represented_vocab_set (rows and columns of the affinity matrix) are labelled as unknown (-1).</p>

In [55]:
"""
Handles processing labelled samples.
"""

# Set of our vocab words that are represented in the embedding matrix.
represented_vocab_set = set(word_to_matrix_row.keys())

# Maps each loaded labelled sampled to a unique id for each theme.
word_to_theme_id = {}

# Maps the unique theme id to its actual name.
id_to_theme_name = {-1 : 'Unknown'}

# Looping through all the labelled text files.
for i, file in enumerate(glob("./Labelled-Data/*")):
    with open(file, 'r') as f:
        
        # Extracting the name of the theme from the filename.
        theme_name = file.split('/')[-1][ : -4]
        
        # Maps a unique id to each theme.
        id_to_theme_name[i] = theme_name
        
        # Goes through all words in the file.
        for word in f.read().split("\n"):
            
            # Reformatting text.
            word = word.lower().strip()
            
            # Removing blank lines
            if word == '':
                continue
                
            # Checking that the word that we hand label has an entry
            # in the affinity matrix.
            if word not in represented_vocab_set:
                print('%s is not in the affinity matrix.' % word)
                continue
            
            # Mapping the word to its unique theme id
            word_to_theme_id[word] = i
            
# Set of words that have a label
labelled_words_set = set(word_to_theme_id.keys())

In [56]:
"""
Handles processing unlabelled samples.
"""

# Loops through all words that are represented in the affinity matrix.
for word in represented_vocab_set:
    
    # If a word is not labelled.
    if word not in labelled_words_set:
        
        # Assign it an unknown label (-1).
        word_to_theme_id[word] = -1

<h1>MultiRank Walk</h1>

<p>***magic***</p>

<p>How does this work?</p>
<p>¯\_(ツ)_/¯</p>

In [32]:
"""
Initializing variables needed for the MRW.
"""

num_data_points = len(word_to_theme_id)
represented_vocab_set = set(word_to_matrix_row.keys())

In [ ]:

# Using the affinity matrix to calculate the diagonal matrix D 
# by summing the rows of the affinity matrix and
# putting then on the diagonal.
D = np.diag([np.sum(x) for x in affinity_matrix])

# Using the affinity matrix and D to calculate the weighted transition matrix W.
W = np.zeros((num_data_points, num_data_points))

# Looping through all elements in the affinity matrix
for row in range(num_data_points):
    
    # Logging.
    sys.stdout.write("\r%d/%d" % (row + 1, num_data_points))
    
    for col in range(num_data_points):
        W[row][col] = affinity_matrix[row][col] / D[row][row]

<h1>Checkpoint</h1>

In [ ]:
np.save('./Checkpoints/W', W)

<h1>Continuing MRW</h1>

In [33]:
W = np.load('./Checkpoints/W.npy')

In [59]:
# Populating list of u vectors.
u_vectors = []

# Seeding all u vectors
# The length of the dictionary contains the number of known
# classes plus one for the unkown. This loop will go through
# all known classes.
# There will be one u vector for each class.
for class_id in range(len(id_to_theme_name.keys()) - 1):

    # Initializing a u vector to the same size of indices as there are
    # represented words in the affinity matrix
    u = np.zeros((num_data_points, 1))
    
    # For each u vector, change the index to 1 at the points
    # where a word of this class_id appears in the affinity matrix.
    # 
    # This loops through all the words represented in the affinity matrix.
    for word in represented_vocab_set:
        
        # If a word belong to this class_id.
        if word_to_theme_id[word] == class_id:
            
            # Update the u vector at the same index at which the word
            # appears in the affinity matrix.
            u[word_to_matrix_row[word]][0] = 1
    
    # Generating matrix of u vectors.
    u_vectors.append(u)

# Normalizing each u vector.
for i in range(len(u_vectors)):
    norm_factor = np.linalg.norm(u_vectors[i], ord=1)
    u_vectors[i] /= norm_factor

In [60]:
"""
Finding the ranking vectors r.
"""

r_vectors = []

# Initializing r vectors.
for _ in u_vectors:
    
    r_vectors.append(np.ones((num_data_points, 1)))

In [61]:
# Hyperparameters
num_iterations = 100
damping = 0.95

# Looping until convergence.
for iteration in range(num_iterations):
    
    sys.stdout.write('\r%d/%d' % (iteration + 1, num_iterations))
    
    # Setting new values for each r/u vector.
    for i in range(len(r_vectors)):
        r_vectors[i] = (1 - damping) * u_vectors[i] + np.transpose(damping * np.matmul(np.transpose(r_vectors[i]), W))

100/100

<h1>Checkpoint</h1>

In [206]:
np.save('./Checkpoints/r_vectors', r_vectors)

<h1>Evaluation</h1>

In [63]:
r_vectors = np.load('./Checkpoints/r_vectors.npy')
word_to_matrix_row = np.load('./Checkpoints/word_to_matrix_row.npy', allow_pickle=True).item()
represented_vocab_set = set(word_to_matrix_row.keys())

In [199]:
def predict_category(word):
    
    options = []
    word_idx = word_to_matrix_row[word]

    for r in r_vectors:
        options.append(r[word_idx])

    largest = max(options)
    smallest = min(options)
    
    
    ret = None
    
    for i, option in enumerate(options):
        if option == largest:            
            ret = id_to_theme_name[i]
            break
        
    return ret

In [197]:
def generate_prediction_for_text(text):
    
    vocab = process_text(text)
    
    categories = []
    
    for word in vocab:
        
        if word in represented_vocab_set:
            category = predict_category(word)
            categories.append(category)
            
    print("=" * 20)
            
    # print(categories)
    for category in set(categories):
        print("%s : %.2f%%" % (category, (categories.count(category) / len(categories)) * 100))
        
    print("=" * 20)

In [190]:
def generate_prediction_for_file(file_path):
    
    print("=" * 20)
    print("%s" % file_path.split("/")[-1][:-4])
    print("=" * 20)
    
    raw = ""
    
    with open(file_path, 'r') as f:
        raw += f.read().replace('\n', ' ')
    
    generate_prediction_for_text(raw)
    
    print("\n")

In [ ]:
base_path = './English-Writers/*'

for author_path in glob("./English-Writers/*"):
    
    author_name = author_path.split('/')[-1]
    
    print("*" * 30)
    print(author_name)
    print("*" * 30)
    
    for text_file_path in glob('./English-Writers/%s/*.txt' % author_name):
        generate_prediction_for_file(text_file_path)